# Notes
- Use pip freeze to generate requirements.txt
- Group data for each year (SQL select) into a dataframe

# Requirements

* Query the dataset using sqlite. Only load the final dataset into a dataframe. ✅

* Give an overview of the respondents of the survey. What is the sample size? ✅
* What are the sociodemographic features of the respondents? Do you see any evidence of sampling bias? ✅
* Perform exploratory data analysis. This should include creating statistical summaries and charts, checking for correlations and other relationships between variables, as well as other EDA elements. ✅
* In a plot, report the prevalence rate of at least three mental diseases. (https://en.wikipedia.org/wiki/Prevalence) ✅
* Make sure to plot the confidence interval and provide its interpretation. ✅
* Your notebook should be readable as a standalone document. In Markdown cells inform the reader of the questions you are trying to answer, and provide an interpretation of your results.
* Provide suggestions about how your analysis can be improved.
# Questions to answer (general)

- What are the main types and subtypes of data?
- What are the main metrics of location? What are their main characteristics?
- What is variability? What are the main metrics of variability and their characteristics?
- What is a confidence interval? Why do we need it? Why is it not sufficient to just report the point estimates?
- What is correlation? How do we use it to analyze data?
- What is a contingency table?

# Plan of action

- Import data into a single dataframe, that is coherent (it makes sense looking at it)
- Review the data
- Clean the data
- Perform exploratory data analysis, main goal

Let's filter the data to only include the questions that are present in all years, as we are interested in the trends over time.
Also, let's clean the data by renaming the columns to lowercase and removing spaces, and renaming SurveyId to year as it is more intuitive.

In [37]:
import sqlite3
import pandas as pd
import plotly.express as px
import helpers
import numpy as np

conn = sqlite3.connect('mental_health.sqlite')

query = """
SELECT 
    s.SurveyID as year,
    s.Description as survey_description,
    a.UserID as user_id,
    a.QuestionID as question_id,
    q.QuestionText as question_text,
    a.AnswerText as answer_text
FROM Answer a
JOIN Question q ON a.QuestionID = q.QuestionID
JOIN Survey s ON a.SurveyID = s.SurveyID
"""

df = pd.read_sql_query(query, conn)

conn.close()

df.columns = df.columns.str.lower()

df['answer_text'] = df['answer_text'].str.lower()

df.head()

,year,survey_description,user_id,question_id,question_text,answer_text
0,2014,mental health survey for 2014,1,1,What is your age?,37
1,2014,mental health survey for 2014,2,1,What is your age?,44
2,2014,mental health survey for 2014,3,1,What is your age?,32
3,2014,mental health survey for 2014,4,1,What is your age?,31
4,2014,mental health survey for 2014,5,1,What is your age?,31


In [38]:
# Clean age data by removing invalid values like -1, <18 and mistakes like 99 (we assume it's a mistake)
age_df = df[df['question_text'] == 'What is your age?']
clean_age = pd.to_numeric(age_df['answer_text'], errors='coerce')
clean_age = clean_age[
    (clean_age >= 18) & (
            clean_age <= 80)]
clean_age_stats = clean_age.describe()

age_df

,year,survey_description,user_id,question_id,question_text,answer_text
0,2014,mental health survey for 2014,1,1,What is your age?,37
1,2014,mental health survey for 2014,2,1,What is your age?,44
2,2014,mental health survey for 2014,3,1,What is your age?,32
3,2014,mental health survey for 2014,4,1,What is your age?,31
4,2014,mental health survey for 2014,5,1,What is your age?,31
...,...,...,...,...,...,...
203936,2019,mental health survey for 2019,4214,1,What is your age?,27
203937,2019,mental health survey for 2019,4215,1,What is your age?,48
203938,2019,mental health survey for 2019,4216,1,What is your age?,50
203939,2019,mental health survey for 2019,4217,1,What is your age?,30


Let's also look at age distribution by year

In [39]:
# Prepare the data
age_df_clean = age_df.copy()
age_df_clean['clean_age'] = pd.to_numeric(age_df_clean['answer_text'], errors='coerce')
age_df_clean = age_df_clean[(age_df_clean['clean_age'] >= 18) & (age_df_clean['clean_age'] <= 100)]

# Create statistical summary by year
yearly_age_stats = age_df_clean.groupby('year')['clean_age'].describe()
overall_stats = age_df_clean['clean_age'].describe()

# Create DataFrame for statistics including overall summary
yearly_stats_df = yearly_age_stats.reset_index()
overall_row = pd.DataFrame([{'year': 'All Years'} | overall_stats.to_dict()], index=[len(yearly_stats_df)])
yearly_stats_df = pd.concat([yearly_stats_df, overall_row], ignore_index=True)
yearly_stats_df = yearly_stats_df.round(2)

# Create distribution plot
fig_dist = px.histogram(
    age_df_clean,
    x='clean_age',
    title='Age Distribution in Tech Industry',
    labels={'clean_age': 'Age', 'count': 'Frequency'},
    marginal='violin',
    nbins=30,
    height=400
)

fig_dist.update_layout(
    showlegend=False,
    template='plotly_white'
)

# Create time series plot
plot_df = yearly_stats_df[yearly_stats_df['year'] != 'All Years'].copy()

fig_time = px.line(
    plot_df,
    x='year',
    y=['mean', '25%', '50%', '75%'],
    title='Age Metrics Over Time',
    labels={
        'value': 'Age',
        'year': 'Year',
        'variable': 'Metric'
    },
    markers=True,
    height=500
)

# Update line names
fig_time.update_traces(name='Mean Age', selector=dict(name='mean'))
fig_time.update_traces(name='25th Percentile', selector=dict(name='25%'))
fig_time.update_traces(name='Median (50th)', selector=dict(name='50%'))
fig_time.update_traces(name='75th Percentile', selector=dict(name='75%'))

# Update layout
fig_time.update_layout(
    xaxis=dict(
        tickmode='array',
        ticktext=plot_df['year'].astype(int).astype(str),
        tickvals=plot_df['year']
    ),
    hovermode='x unified',
    legend_title='Age Metrics',
    template='plotly_white'
)

# Add sample size annotations
annotations = []
for idx, row in plot_df.iterrows():
    annotations.append(
        dict(
            x=row['year'],
            y=row['mean'],
            text=f"n={row['count']:.0f}",
            showarrow=False,
            yshift=20,
            font=dict(size=10)
        )
    )
fig_time.update_layout(annotations=annotations)

# Update hover template
fig_time.update_traces(
    hovertemplate="<br>".join([
        "Year: %{x}",
        "%{name}: %{y:.1f} years",
        "<extra></extra>"
    ])
)

# Display plots and statistics
fig_dist.show()
fig_time.show()
yearly_stats_df.round(2)

,year,count,mean,std,min,25%,50%,75%,max
0,2014,1252.0,32.08,7.29,18.0,27.0,31.0,36.0,72.0
1,2016,1429.0,34.13,8.26,19.0,28.0,33.0,39.0,99.0
2,2017,754.0,34.99,8.34,18.0,29.0,34.0,40.0,67.0
3,2018,417.0,34.92,8.05,19.0,29.0,34.0,39.0,67.0
4,2019,351.0,35.60,8.89,19.0,29.0,34.0,41.0,64.0
5,All Years,4203.0,33.88,8.13,18.0,28.0,33.0,38.0,99.0


### Sample Size and Distribution:
- Count: 4203 respondents
- Mean: 33.88 years
- Median (50%): 33 years
- The mean and median being close suggests a relatively symmetric distribution

### Age Spread:

- Standard Deviation: 8.13 years
- IQR: 38 years (75th) - 28 years (25th) = 10 years
- Range: 18 years (min) to 99 years (max)

### Evidence of Sampling Bias:

- Age concentration: 50% of respondents are between 28-38 years
- Under-representation of:
    - Senior tech workers (40+ years)
    - Early career professionals (< 25 years)
    - The narrow standard deviation (8.13 years) suggests limited age diversity

## Participation Bias:

- Sharp decline in participation from 2016 to 2019
- Could affect trend analysis reliability
- More recent years might not be as representative

Let's analyze other demographics to get a fuller picture.

In [40]:
# Apply the categorization
gender_df = df[df['question_text'] == 'What is your gender?'].copy()
gender_df['category'] = gender_df['answer_text'].apply(helpers.categorize_gender)

# Create the distribution
gender_distribution = gender_df.groupby(['year', 'category']).size().unstack(fill_value=0)

# Calculate percentages
gender_distribution_pct = gender_distribution.div(gender_distribution.sum(axis=1), axis=0) * 100

# Show both counts and percentages
gender_distribution

category,Female,Male,Other
year,,,
2014,247,991,22
2016,336,1057,40
2017,218,502,36
2018,125,266,26
2019,98,228,26


In [41]:
gender_distribution_pct

category,Female,Male,Other
year,,,
2014,19.603175,78.650794,1.746032
2016,23.447313,73.761340,2.791347
2017,28.835979,66.402116,4.761905
2018,29.976019,63.788969,6.235012
2019,27.840909,64.772727,7.386364


In [42]:
# Location/Country distribution
location_dist = helpers.get_responses_by_question(df, 'What country do you live in?')
location_dist.sum().nlargest(10)

answer_text
united states of america    1853
united states                751
united kingdom               482
canada                       199
germany                      136
netherlands                   98
australia                     73
france                        51
ireland                       51
india                         50
dtype: int64

In [43]:
# Company size distribution
company_size_dist = helpers.get_responses_by_question(df, 'How many employees does your company or organization have?')
company_size_dist

answer_text,-1,1-5,100-500,26-100,500-1000,6-25,more than 1000
year,,,,,,,
2014,0,162,176,289,61,290,282
2016,287,60,248,292,80,210,256
2017,113,20,203,128,48,86,158
2018,56,5,81,70,31,69,105
2019,48,7,80,45,27,34,111


Looking at the country distribution, we can identify clear geographic sampling biases:

Strong US Dominance:


United States total: 2,604 respondents (1,853 + 751 from different labelings)
This represents approximately 65% of respondents


### English-Speaking Countries Bias:


Top English-speaking countries:

USA: 2,604 respondents
UK: 482 respondents
Canada: 199 respondents
Australia: 73 respondents
Ireland: 51 respondents

Western Europe Representation:

Moderate representation from:

Germany: 136 respondents
Netherlands: 98 respondents
France: 51 respondents

## Underrepresentation:


Only one Asian country in top 10 (India: 50 respondents)
No representation from:

South America
Africa
Most of Asia
Eastern Europe

### Sampling Biases to Consider:

Language Barrier: Survey likely conducted in English
Distribution Channels: Survey might have been distributed through US-centric networks

This geographic distribution limits our ability to make global generalizations about:

- Mental health in tech globally
- Cultural differences in mental health approaches
- Regional workplace practices

Looking at the company size distribution across years, let's analyze the patterns:

### Overall Distribution Pattern:

Very diverse representation from small to large companies
Strong representation from both ends:

Small companies (1-5, 6-25 employees)
Large enterprises (More than 1000 employees)

Good representation of mid-sized companies (26-100, 100-500)

### Company Size Bias:

Good representation of different company sizes

### Missing Data:

Significant "-1" values in 2016-2019
Could affect analysis reliability

Changes Over Time:

Declining participation across all company sizes


# Mental Health Conditions Prevalence Analysis

Let's analyze the prevalence of mental health conditions in the tech industry.


In [44]:
diagnosis_responses = df[df['question_text'] == "If yes, what condition(s) have you been diagnosed with?"]

conditions = [
    "Anxiety Disorder",
    "Mood Disorder",
    "Attention Deficit Hyperactivity Disorder"
]

conditions_plot_data = []
total_respondents = df['user_id'].nunique()

for condition in conditions:
    condition_count = diagnosis_responses[
        diagnosis_responses['answer_text'].str.contains(condition, na=False, case=False)
    ]['user_id'].nunique()

    prevalence = (condition_count / total_respondents) * 100

    # Calculate confidence intervals
    z = 1.96  # 95% confidence level
    n = total_respondents
    p = prevalence / 100

    ci_lower = ((p + z * z / (2 * n) - z * np.sqrt((p * (1 - p) + z * z / (4 * n)) / n)) / (1 + z * z / n)) * 100
    ci_upper = ((p + z * z / (2 * n) + z * np.sqrt((p * (1 - p) + z * z / (4 * n)) / n)) / (1 + z * z / n)) * 100

    conditions_plot_data.append({
        'Condition': condition,
        'Prevalence': prevalence,
        'CI_lower': ci_lower,
        'CI_upper': ci_upper,
        'Count': condition_count
    })

conditions_df = pd.DataFrame(conditions_plot_data)

fig = px.bar(conditions_df,
             x='Condition',
             y='Prevalence',
             error_y=conditions_df.apply(lambda row: {'array': [row['CI_upper'] - row['Prevalence']]}, axis=1),
             error_y_minus=conditions_df.apply(lambda row: {'array': [row['Prevalence'] - row['CI_lower']]}, axis=1),
             title='Prevalence of Mental Health Conditions in Tech Industry (2014)')

fig.update_layout(
    xaxis_title="Condition",
    yaxis_title="Prevalence (%)",
    title_x=0.5,
    template='plotly_white',
    showlegend=False,
    xaxis_tickangle=-45
)

fig.update_traces(
    hovertemplate="<br>".join([
        "<b>%{x}</b>",
        "Prevalence: %{y:.1f}%",
        "95% CI: (%{customdata[0]:.1f}% - %{customdata[1]:.1f}%)",
        "Count: %{customdata[2]}",
        "<extra></extra>"
    ]),
    customdata=conditions_df[['CI_lower', 'CI_upper', 'Count']]
)

fig.show()

print("\nDetailed Statistics:")
for _, row in conditions_df.iterrows():
    print(f"\n{row['Condition']}:")
    print(f"Prevalence: {row['Prevalence']:.1f}%")
    print(f"95% CI: ({row['CI_lower']:.1f}% - {row['CI_upper']:.1f}%)")
    print(f"Count: {row['Count']}")


Detailed Statistics:

Anxiety Disorder:
Prevalence: 8.2%
95% CI: (7.4% - 9.0%)
Count: 345

Mood Disorder:
Prevalence: 9.8%
95% CI: (8.9% - 10.7%)
Count: 412

Attention Deficit Hyperactivity Disorder:
Prevalence: 2.9%
95% CI: (2.4% - 3.4%)
Count: 121


In [48]:
import plotly.express as px
import pandas as pd
import numpy as np

# Filter for diagnosis responses
diagnosis_responses = df[df['question_text'] == "If yes, what condition(s) have you been diagnosed with?"]

# Common conditions to look for
conditions = [
    "Anxiety Disorder",
    "Mood Disorder",
    "Attention Deficit Hyperactivity Disorder"
]

# Calculate total number of respondents who answered the diagnosis question
total_respondents = len(diagnosis_responses[diagnosis_responses['answer_text'] != '-1'])

conditions_plot_data = []

for condition in conditions:
    # Count users with each condition (excluding -1 responses)
    condition_count = diagnosis_responses[
        (diagnosis_responses['answer_text'].str.contains(condition, na=False, case=False)) &
        (diagnosis_responses['answer_text'] != '-1')
        ]['user_id'].nunique()

    # Calculate prevalence
    prevalence = (condition_count / total_respondents) * 100

    # Calculate Wilson score intervals
    z = 1.96  # 95% confidence level
    n = total_respondents
    p = prevalence / 100

    ci_lower = ((p + z * z / (2 * n) - z * np.sqrt((p * (1 - p) + z * z / (4 * n)) / n)) / (1 + z * z / n)) * 100
    ci_upper = ((p + z * z / (2 * n) + z * np.sqrt((p * (1 - p) + z * z / (4 * n)) / n)) / (1 + z * z / n)) * 100

    conditions_plot_data.append({
        'Condition': condition,
        'Prevalence': prevalence,
        'CI_lower': ci_lower,
        'CI_upper': ci_upper,
        'Count': condition_count
    })

# Convert to DataFrame
conditions_df = pd.DataFrame(conditions_plot_data)

# Create the plot
fig = px.bar(conditions_df,
             x='Condition',
             y='Prevalence',
             error_y=conditions_df.apply(lambda row: {'array': [row['CI_upper'] - row['Prevalence']]}, axis=1),
             error_y_minus=conditions_df.apply(lambda row: {'array': [row['Prevalence'] - row['CI_lower']]}, axis=1),
             title='Prevalence of Mental Health Conditions with 95% Confidence Intervals')

# Update layout
fig.update_layout(
    xaxis_title="Condition",
    yaxis_title="Prevalence (%)",
    title_x=0.5,
    template='plotly_white',
    showlegend=False,
    xaxis_tickangle=-45
)

# Add hover template
fig.update_traces(
    hovertemplate="<br>".join([
        "<b>%{x}</b>",
        "Prevalence: %{y:.1f}%",
        "95% CI: (%{customdata[0]:.1f}% - %{customdata[1]:.1f}%)",
        "Count: %{customdata[2]}",
        "<extra></extra>"
    ]),
    customdata=conditions_df[['CI_lower', 'CI_upper', 'Count']]
)

# Show plot
fig.show()

# Print detailed statistics
print("\nDetailed Statistics:")
for _, row in conditions_df.iterrows():
    print(f"\n{row['Condition']}:")
    print(f"Count: {row['Count']}")
    print(f"Prevalence: {row['Prevalence']:.1f}%")
    print(f"95% CI: ({row['CI_lower']:.1f}% - {row['CI_upper']:.1f}%)")


Detailed Statistics:

Anxiety Disorder:
Count: 345
Prevalence: 28.6%
95% CI: (26.1% - 31.2%)

Mood Disorder:
Count: 412
Prevalence: 34.1%
95% CI: (31.5% - 36.9%)

Attention Deficit Hyperactivity Disorder:
Count: 121
Prevalence: 10.0%
95% CI: (8.5% - 11.8%)
